In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/colab_libs')

Mounted at /content/drive


In [ ]:
! pip install mne
! pip install scipy
! pip install mne
import numpy as np
import scipy.io
import mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.1 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
import numpy as np
import scipy.io as sio
import mne

def mat_to_raw(fname):
    mat   = sio.loadmat(fname, squeeze_me=True, struct_as_record=False)
    EEG   = mat['EEG']
    data  = np.asarray(EEG.data) * 1e-6   # μV -> V
    sfreq = float(EEG.srate)
    ch_names = [c.labels for c in np.atleast_1d(EEG.chanlocs)]
    info = mne.create_info(ch_names, sfreq, ch_types='eeg')
    return mne.io.RawArray(data, info, verbose=False)

def events_from_mat(fname):
    m   = sio.loadmat(fname, squeeze_me=True, struct_as_record=False)
    EEG = m['EEG']
    evs = []
    for ev in np.atleast_1d(EEG.event):
        sample = int(round(ev.latency)) - 1   # EEGLAB 多为 1-based
        code   = int(ev.type)
        evs.append([sample, 0, code])
    return np.asarray(evs, dtype=int)

def add_stim(raw, events, name='STI'):
    stim = np.zeros(raw.n_times, float)
    stim[events[:, 0]] = events[:, 2]
    info = mne.create_info([name], raw.info['sfreq'], ['stim'])
    raw.add_channels([mne.io.RawArray(stim[None, :], info)], force_update_info=True)


def concat_nr_tsr(nr_folder, tsr_folder):

    files = []
    files += sorted(Path(nr_folder).glob("*NR*_EEG.mat"))
    files += sorted(Path(tsr_folder).glob("*TSR*_EEG.mat"))


    raws = []
    dropped = []

    for f in files:
      r = mat_to_raw(f)
      ev = events_from_mat(f)
      add_stim(r, ev, name='STI')
      r_copy = r.copy()

      r_copy.filter(l_freq=2., h_freq=None, picks="eeg", verbose=False)

      data_eeg = r_copy.get_data(picks="eeg")
      std = data_eeg.std(axis=1, keepdims=True)

      mask = (np.abs(data_eeg) <= 3 * std).all(axis=0)
      data_eeg_clean = data_eeg[:, mask]

      sd = data_eeg_clean.std()
      if sd < 1e-4:   # 100 µV = 1e-4 V
          raws.append(r)
      else:
          dropped.append(f.name)

    print("fiels deleted:", dropped if dropped else "no fiels dropped")



    raw_all = mne.concatenate_raws(raws)
    print(f"concatenated: {len(raws)} files, total times ≈ {raw_all.n_times/raw_all.info['sfreq']:.1f}s")

    events = mne.find_events(raw_all, stim_channel='STI', shortest_event=1)
    return raw_all, events, dropped


# pplied the ±3SD time-point mask to the STI channel as wel
# That truncated some stimulus pulses to just 1 sample.
# mne.find_events expects each event to be at least 2 samples long by default


In [ ]:
from pathlib import Path

nr_root  = "/content/drive/MyDrive/Zuco2/task1 - NR/Raw data"
tsr_root = "/content/drive/MyDrive/Zuco2/task2 - TSR/Raw data"

subjects = sorted([d.name for d in Path(nr_root).iterdir() if d.is_dir()])

subj_raws = {}
all_dropped_files = []

for sub in subjects:
  print(f"\nprocessing {sub} ...")
  raw_all, events, dropped = concat_nr_tsr(Path(nr_root)/sub, Path(tsr_root)/sub)


  subj_raws[sub] = raw_all

  all_dropped_files.extend(dropped)

print(f"\nDropped in total {len(all_dropped_files)} files：")
for fname in all_dropped_files:
  print("  -", fname)



processing YAC ...
Creating RawArray with float64 data, n_channels=1, n_times=225411
    Range : 0 ... 225410 =      0.000 ...   450.820 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=157559
    Range : 0 ... 157558 =      0.000 ...   315.116 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=133507
    Range : 0 ... 133506 =      0.000 ...   267.012 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=106505
    Range : 0 ... 106504 =      0.000 ...   213.008 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=108049
    Range : 0 ... 108048 =      0.000 ...   216.096 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=97277
    Range : 0 ... 97276 =      0.000 ...   194.552 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=118755
    Range : 0 ... 118754 =      0.000 ...   237.508 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=170233
    Ra

In [ ]:
out_root = Path("/content/drive/MyDrive/Zuco2/processed_raws")
out_root.mkdir(exist_ok=True)

for sub, raw in subj_raws.items():
  out_path = out_root / f"{sub}_concat_raw.fif"
  raw.save(out_path, overwrite=True)
  print(f"Saved: {out_path}")

Writing /content/drive/MyDrive/Zuco2/processed_raws/YAC_concat_raw.fif
Closing /content/drive/MyDrive/Zuco2/processed_raws/YAC_concat_raw.fif
[done]
Saved: /content/drive/MyDrive/Zuco2/processed_raws/YAC_concat_raw.fif
Writing /content/drive/MyDrive/Zuco2/processed_raws/YAG_concat_raw.fif
Closing /content/drive/MyDrive/Zuco2/processed_raws/YAG_concat_raw.fif
[done]
Saved: /content/drive/MyDrive/Zuco2/processed_raws/YAG_concat_raw.fif
Writing /content/drive/MyDrive/Zuco2/processed_raws/YAK_concat_raw.fif
Closing /content/drive/MyDrive/Zuco2/processed_raws/YAK_concat_raw.fif
[done]
Saved: /content/drive/MyDrive/Zuco2/processed_raws/YAK_concat_raw.fif
Writing /content/drive/MyDrive/Zuco2/processed_raws/YDG_concat_raw.fif
Closing /content/drive/MyDrive/Zuco2/processed_raws/YDG_concat_raw.fif
[done]
Saved: /content/drive/MyDrive/Zuco2/processed_raws/YDG_concat_raw.fif
Writing /content/drive/MyDrive/Zuco2/processed_raws/YDR_concat_raw.fif
Closing /content/drive/MyDrive/Zuco2/processed_raws/Y